In [9]:
import pandas as pd

# First look
df = pd.read_excel('./data/base_de_dados_desafio.xlsx')
df

,ID_CHAMADA,DATA_CONTATO,DURACAO_SEG_CHAMADA,OPERADOR,EMPRESA,CLASSIFICACAO,CLASSIFICACAO_DETALHE
0,CHAMADA 0001,2021-06-01 18:52:00.000,281,OPERADOR 001,EMPRESA 01,ARGUMENTAÇÃO,NEGATIVAÇÃO
1,CHAMADA 0001,2021-06-01 18:52:00.000,281,OPERADOR 001,EMPRESA 01,PROPOSTA,PRAZO PARA PAGAMENTO
2,CHAMADA 0001,2021-06-01 18:52:00.000,281,OPERADOR 001,EMPRESA 01,PROPOSTA,DESCONTO A VISTA
3,CHAMADA 0001,2021-06-01 18:52:00.000,281,OPERADOR 001,EMPRESA 01,PROPOSTA,DESCONTO A VISTA
4,CHAMADA 0001,2021-06-01 18:52:00.000,281,OPERADOR 001,EMPRESA 01,PROPOSTA,DESCONTO A VISTA
...,...,...,...,...,...,...,...
34515,CHAMADA 2160,2022-03-16 08:36:52.000,292,OPERADOR 015,EMPRESA 02,CHAMADA COM ACORDO,CHAMADA COM ACORDO
34516,CHAMADA 2150,2022-03-15 18:49:59.000,112,OPERADOR 489,EMPRESA 02,CHAMADA COM ACORDO,CHAMADA COM ACORDO
34517,CHAMADA 2150,2022-03-15 18:49:59.000,112,OPERADOR 489,EMPRESA 02,CHAMADA COM ACORDO,CHAMADA COM ACORDO
34518,CHAMADA 2150,2022-03-15 18:49:59.000,112,OPERADOR 489,EMPRESA 02,CHAMADA COM ACORDO,CHAMADA COM ACORDO


In [10]:
# 1NF
# Check if the row value has meaninfull information
def check_prefix(df,col_names:list):
    "Print the first element of the column splited by ' ' (space)"
    for col in col_names:
        prefix = df[col].str.split(' ').str.get(0)
        print(f'{col} prefixs:\n{set(prefix.tolist())}\n')

prefix_cols = ['ID_CHAMADA', 'OPERADOR', 'EMPRESA']
check_prefix(df, prefix_cols)


ID_CHAMADA prefixs:
{'CHAMADA'}

OPERADOR prefixs:
{'OPERADOR'}

EMPRESA prefixs:
{'EMPRESA'}



In [11]:
# Prefix does not have meaninfull info, so we will split and keep the numbers
def transform_sufix(df,col_names:list):
    "Takes the second element of the column and convert it to int"
    for col in col_names:
        df[col] = df[col].str.split(' ').str.get(1).astype(int)

transform_sufix(df,prefix_cols)

df.head()

,ID_CHAMADA,DATA_CONTATO,DURACAO_SEG_CHAMADA,OPERADOR,EMPRESA,CLASSIFICACAO,CLASSIFICACAO_DETALHE
0,1,2021-06-01 18:52:00.000,281,1,1,ARGUMENTAÇÃO,NEGATIVAÇÃO
1,1,2021-06-01 18:52:00.000,281,1,1,PROPOSTA,PRAZO PARA PAGAMENTO
2,1,2021-06-01 18:52:00.000,281,1,1,PROPOSTA,DESCONTO A VISTA
3,1,2021-06-01 18:52:00.000,281,1,1,PROPOSTA,DESCONTO A VISTA
4,1,2021-06-01 18:52:00.000,281,1,1,PROPOSTA,DESCONTO A VISTA


In [12]:
# 2NF
# ID_CHAMADA can have only one DATA_CONTATO, DURACAO_SEG_CHAMADA, OPERADOR, EMPRESA
dim_chamada = df[['ID_CHAMADA', 'DATA_CONTATO', 'DURACAO_SEG_CHAMADA', 'OPERADOR', 'EMPRESA']]
dim_chamada = dim_chamada.drop_duplicates()
display(dim_chamada)


# Columns CLASSIFICACAO, CLASSIFICACAO_DETALHE have direct dependency
fct_chamada = df[['ID_CHAMADA', 'CLASSIFICACAO', 'CLASSIFICACAO_DETALHE']]

# There are a lot of duplication and it can be meningfull
# Let's add a count column and use the summery table
fct_chamada = df.pivot_table(index = ['ID_CHAMADA', 'CLASSIFICACAO', 'CLASSIFICACAO_DETALHE'], aggfunc ='size')
fct_chamada = fct_chamada.reset_index()
fct_chamada = fct_chamada.rename(columns={0 : 'COUNT_COMPORTAMENTO'})
display(fct_chamada)

,ID_CHAMADA,DATA_CONTATO,DURACAO_SEG_CHAMADA,OPERADOR,EMPRESA
0,1,2021-06-01 18:52:00.000,281,1,1
14,2,2021-06-01 18:51:00.000,158,2,1
27,3,2021-06-01 18:49:00.000,421,3,1
31,4,2021-06-01 18:46:00.000,218,4,1
42,5,2021-06-01 18:46:00.000,211,5,1
...,...,...,...,...,...
34271,2174,2022-03-16 09:24:22.000,184,545,2
34340,2175,2022-03-16 14:31:08.000,326,15,2
34363,2176,2022-03-16 13:56:42.000,181,485,2
34458,2177,2022-03-14 19:37:03.000,141,546,2


,ID_CHAMADA,CLASSIFICACAO,CLASSIFICACAO_DETALHE,COUNT_COMPORTAMENTO
0,1,ARGUMENTAÇÃO,AÇÕES DE COBRANÇA,1
1,1,ARGUMENTAÇÃO,DESCONEXÃO DA LINHA,2
2,1,ARGUMENTAÇÃO,NEGATIVAÇÃO,1
3,1,CHAMADA COM ACORDO,CHAMADA COM ACORDO,3
4,1,PROPOSTA,DESCONTO A VISTA,6
...,...,...,...,...
9453,2175,CHAMADA COM ACORDO,CHAMADA COM ACORDO,4
9454,2175,SONDAGEM,AVALIACAO FINANCEIRA,1
9455,2176,ARGUMENTAÇÃO,AÇÕES DE COBRANÇA,3
9456,2177,CHAMADA COM ACORDO,CHAMADA COM ACORDO,1


**Observação**

Em um contexto real as colunas EMPRESA e OPERADOR teriam informações adicionais como _nome_, _idade_, _endereço_, _data de criação_, etc.
Com isso eu manteria os id's deles na tabela fato (fct) e montaria tabelas dimensão para EMPRESA e OPERADOR seguindo a modelagem *star schema* de Kimble.

In [13]:
# Adding to each chamada if it was successeful or not
df_stg_sucesso = fct_chamada[(fct_chamada['CLASSIFICACAO'] == 'CHAMADA COM ACORDO')]
df_stg_sucesso

dim_chamada = dim_chamada.assign(SUCESSO=dim_chamada['ID_CHAMADA'].
                                 isin(df_stg_sucesso['ID_CHAMADA']).
                                 astype(bool))
dim_chamada

,ID_CHAMADA,DATA_CONTATO,DURACAO_SEG_CHAMADA,OPERADOR,EMPRESA,SUCESSO
0,1,2021-06-01 18:52:00.000,281,1,1,True
14,2,2021-06-01 18:51:00.000,158,2,1,True
27,3,2021-06-01 18:49:00.000,421,3,1,True
31,4,2021-06-01 18:46:00.000,218,4,1,True
42,5,2021-06-01 18:46:00.000,211,5,1,False
...,...,...,...,...,...,...
34271,2174,2022-03-16 09:24:22.000,184,545,2,True
34340,2175,2022-03-16 14:31:08.000,326,15,2,True
34363,2176,2022-03-16 13:56:42.000,181,485,2,False
34458,2177,2022-03-14 19:37:03.000,141,546,2,True


In [14]:
# Saving tables
dim_chamada.to_csv('./data/dim_chamada.csv', index = False)
print('dim_chamada saved!')
fct_chamada.to_csv('./data/fct_chamada.csv', index = False)
print('fct_chamada saved!')


dim_chamada saved!
fct_chamada saved!
